In [2]:
# Custom Modules import
from util.data_loader import load_competitions, load_matches
from util.league_table import generate_league_table
from util.elo_calculator import initialize_team_stats, calculate_elo_ratings, loss_from_comparing_tables, elo_grid_search

from statsbombpy import sb
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

%load_ext autoreload 
%autoreload 2

# Nuanced Team Strength Model

### Used Features to quantify Team Stregnth:
- Possession
- Shots on Goal/on Target
- Passes 
- Duels
- 


### Other Features used to predict Match Outcomes:
- Last 5 matches
- Starting XI
- Stakes of the match 
- Injured/Penalized players
- 

In [11]:
events = sb.events(match_id=3890259)
events.shape

(3691, 91)

In [8]:
distinct_types = events['type'].unique()
print(distinct_types)

['Starting XI' 'Half Start' 'Pass' 'Ball Receipt*' 'Carry' 'Pressure'
 'Duel' 'Ball Recovery' 'Dispossessed' 'Dribble' 'Block' 'Miscontrol'
 'Clearance' 'Shot' 'Goal Keeper' 'Interception' 'Foul Committed'
 'Foul Won' 'Dribbled Past' '50/50' 'Shield' 'Half End' 'Injury Stoppage'
 'Player Off' 'Player On' 'Substitution' 'Tactical Shift']


In [32]:
print(events['pass_outcome'].unique())
print(events['pass_type'].unique())
print(events['pass_height'].unique())

[nan 'Incomplete' 'Out' 'Unknown' 'Pass Offside']
[nan 'Kick Off' 'Throw-in' 'Recovery' 'Free Kick' 'Goal Kick' 'Corner'
 'Interception']
[nan 'Ground Pass' 'High Pass' 'Low Pass']


In [6]:
won_duels = events[
    (events['type'] == 'Duel') |
     #(events['duel_outcome'] == 'Success In Play') |
    (events['type'] == '50/50')
].dropna(axis=1, how='all')

won_duels

,50_50,counterpress,duel_outcome,duel_type,duration,id,index,location,match_id,minute,...,possession,possession_team,possession_team_id,related_events,second,team,team_id,timestamp,type,under_pressure
3802,NaN,NaN,NaN,Aerial Lost,0.000000,939df374-1cdc-4c9f-b84e-fc6dae368909,14,"[79.4, 74.7]",3890261,0,...,2,Borussia Mönchengladbach,185,[347297bb-ed92-4b97-8437-0b6f611b3337],6,Borussia Mönchengladbach,185,00:00:06.338,Duel,True
3803,NaN,NaN,Success In Play,Tackle,0.000000,7fb9b7f9-599b-42ff-8757-8c73f8b3c211,21,"[46.8, 5.0]",3890261,0,...,2,Borussia Mönchengladbach,185,[71bbc4b4-9c51-4114-94c1-7f4792ac8caa],11,Borussia Dortmund,180,00:00:11.121,Duel,True
3804,NaN,True,Won,Tackle,0.000000,ddbb0412-677f-4a78-8ecf-8b5891d91044,27,"[69.5, 67.3]",3890261,0,...,2,Borussia Mönchengladbach,185,[c9d851f1-afca-4f55-a3ce-3bc364fcf2ec],12,Borussia Mönchengladbach,185,00:00:12.965,Duel,True
3805,NaN,True,Success In Play,Tackle,0.000000,d8399bf0-1f73-41be-a0de-6b4d6b9bc11b,112,"[30.9, 56.9]",3890261,1,...,6,Borussia Mönchengladbach,185,[3fe6780b-a04d-497d-a4be-f4165e129fd4],19,Borussia Mönchengladbach,185,00:01:19.944,Duel,True
3806,NaN,NaN,NaN,Aerial Lost,0.000000,3f2350ce-bf37-4f39-a238-e229dae2b699,125,"[50.4, 11.5]",3890261,1,...,6,Borussia Mönchengladbach,185,[0fe70390-2c57-46cb-858c-28ffc045c674],29,Borussia Mönchengladbach,185,00:01:29.210,Duel,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4247,"{'outcome': {'id': 3, 'name': 'Success To Team'}}",NaN,NaN,NaN,0.000000,dd13c22b-cc0d-4b65-967d-a5a6f1d3190b,1598,"[29.3, 18.7]",3890261,32,...,58,Borussia Mönchengladbach,185,[4836489b-493d-45d7-a500-578e099fe154],20,Borussia Dortmund,180,00:32:20.934,50/50,True
4248,"{'outcome': {'id': 1, 'name': 'Lost'}}",NaN,NaN,NaN,0.302487,647d13a2-c2f2-405b-ab77-c0c8afd9cde0,2296,"[63.1, 54.3]",3890261,47,...,86,Borussia Mönchengladbach,185,"[22891964-56d0-4810-ab4d-07c8aa036cdf, b939175...",25,Borussia Mönchengladbach,185,00:02:25.733,50/50,True
4249,"{'outcome': {'id': 3, 'name': 'Success To Team'}}",NaN,NaN,NaN,0.000000,22891964-56d0-4810-ab4d-07c8aa036cdf,2297,"[57.0, 25.8]",3890261,47,...,86,Borussia Mönchengladbach,185,[647d13a2-c2f2-405b-ab77-c0c8afd9cde0],25,Borussia Dortmund,180,00:02:25.733,50/50,True
4250,"{'outcome': {'id': 2, 'name': 'Success To Oppo...",NaN,NaN,NaN,0.000000,ca877304-a309-42f3-a8bd-4d3dc5d25b0f,2412,"[49.1, 12.3]",3890261,51,...,94,Borussia Mönchengladbach,185,[b154fb28-7f1f-4c88-b3fe-9c5fc1bab13e],2,Borussia Dortmund,180,00:06:02.433,50/50,True


In [7]:
won_duels_grouped = events[
    (events['type'] == 'Duel') &
    (events['duel_outcome'] == 'Success In Play')
].dropna(axis=1, how='all').groupby('team').size().reset_index(name='duels_won')

won_duels_grouped

,team,duels_won
0,Borussia Dortmund,8
1,Borussia Mönchengladbach,8


In [35]:
# Calculate completed passes (successful passes with no outcome)
completed_passes = (events[(events['type'] == 'Pass') & 
                         (events['pass_outcome'].isnull())]
                  .groupby('team')
                  .size()
                  .reset_index(name='completed_passes'))

# Calculate total passes
total_passes = (events[events['type'] == 'Pass']
               .groupby('team')
               .size()
               .reset_index(name='total_passes'))

# Merge the two DataFrames
pass_stats = pd.merge(completed_passes, total_passes, on='team')

# Calculate completion percentage
pass_stats['completion_percentage'] = (pass_stats['completed_passes'] / 
                                     pass_stats['total_passes']) * 100

# Sort by completion percentage
pass_stats = pass_stats.sort_values('completion_percentage', ascending=False)

pass_stats.head()

,team,completed_passes,total_passes,completion_percentage
0,Bayern Munich,711,797,89.209536
1,Hamburger SV,150,240,62.500000
